In [119]:
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
import numpy as np

In [120]:
DATA_PATH = "c:\\users\\talibit\\Drug_discovery_data\\data.txt"
DATA_PATH1 = "c:\\users\\talibit\\Drug_discovery_data\\alzheimersdata.txt"
data = open(DATA_PATH, 'r').read().splitlines()
data1 = open(DATA_PATH1, 'r').read().splitlines()

In [121]:
len(data1)

1560

In [122]:
new = []
count =0
for molecule in data:
    count+=1
    if 30 < len(molecule) < 50 and count < 300000:
        if ('T' not in molecule) and ('V' not in molecule) and ('g' not in molecule) and ('L' not in molecule) and ('8' not in molecule):
            new.append(molecule)

In [123]:
new1 = []
count =0
for molecule in data1:
    count+=1
    if 30 < len(molecule) < 50 and count < 300000:
        if ('T' not in molecule) and ('V' not in molecule) and ('g' not in molecule) and ('L' not in molecule) and ('8' not in molecule):
            new1.append(molecule)

In [124]:
len(new1)

847

In [125]:
max_seq_len = len(max(new, key=len))
max_seq_len

49

In [126]:
max_seq_len1 = len(max(new1, key=len))
max_seq_len1

49

In [127]:
l = [['$']+ list(i) for i in new]
padded = pad_sequences(l, maxlen=max_seq_len+1, padding="post", value="£",dtype=object) 
padded_text =["".join(i) for i in padded]
datas = np.array(padded_text)

In [128]:
l1 = [['$']+ list(i) for i in new1]
padded1 = pad_sequences(l1, maxlen=max_seq_len1+1, padding="post", value="£",dtype=object) 
padded_text1 =["".join(i) for i in padded1]
datas1 = np.array(padded_text1)

In [129]:
len(datas1[1])

50

In [130]:
#split data into training validation and test sets (generic data)

seed = 1
np.random.seed(seed)
full_train, test = train_test_split(np.array(datas), test_size=0.2, random_state=seed)

train, val = train_test_split(np.array(full_train), test_size=0.1, random_state=seed)

In [131]:
#split data into training validation and test sets (alzheimer data)

seed = 1
np.random.seed(seed)
full_train1, test1 = train_test_split(np.array(datas1), test_size=0.2, random_state=seed)

train1, val1 = train_test_split(np.array(full_train1), test_size=0.1, random_state=seed)

In [132]:
train = ''.join(train) 
val = ''.join(val) 
# so now train and val are a long string where each chemical string starts with $ and ends with £..£

In [133]:
train1 = ''.join(train1) 
val1 = ''.join(val1) 
# so now train and val are a long string where each chemical string starts with $ and ends with £..£

In [134]:
# this function return for a sequence of chars a tuple of (sequence,next char) where each sequence is a window of seqLen characters
# and the next char is the one following the last char of the sequence
#the sequences are built by moving through the whole sequence by stepSize each time
def n_grams(seqLen, stepSize, data):

    input_chars = [] 
    next_char = [] 
    for i in range(0, len(data) - seqLen, stepSize): 
        input_chars.append(data[i : i + seqLen]) 
        next_char.append(data[i + seqLen]) 

    return input_chars, next_char

# the following function prepare data to feed the neural net
def preparation(data, tokenizer):
    tokenizer.fit_on_texts(data) 
    new_data = tokenizer.texts_to_sequences(data) 

    #N-Grams Sequence 
    seqLen = 15 
    stepSize = 1 
    input_chars, next_char = n_grams(seqLen, stepSize, new_data)
  
    #Assemble Validation Datasets 
    x_data = np.array(input_chars) 
    #x_data.flatten() 
    y_data = np.array(next_char) 
    #y_data_2 = np_utils.to_categorical(y_data) 
    return x_data, y_data

def generate_curves(history, filename):
    fig, (axis1, axis2) = plt.subplots(nrows=1, ncols=2, figsize=(16,6))

    axis1.plot(history.history["accuracy"], label='Train', linewidth=3)
    axis1.plot(history.history["val_accuracy"], label='Validation', linewidth=3)
    axis1.set_title('Model accuracy', fontsize=16, color="white")
    axis1.set_ylabel('accuracy')
    axis1.set_xlabel('epoch')
    axis1.legend(loc='lower right')

    axis2.plot(history.history["loss"], label='Train', linewidth=3)
    axis2.plot(history.history["val_loss"], label='Validation', linewidth=3)
    axis2.set_title('Model loss', fontsize=16, color="white")
    axis2.set_ylabel('loss')
    axis2.set_xlabel('epoch')
    axis2.legend(loc='upper right')
    plt.savefig(filename)

In [135]:
#cheking the n_grams function
tokenizer = Tokenizer(num_words=None, char_level=True, lower=False) 

tokenizer.fit_on_texts(train) 
new_data = tokenizer.texts_to_sequences(train) 


#N-Grams Sequence 
seqLen = 15 
stepSize = 1 
input_chars, next_char = n_grams(seqLen, stepSize, new_data)

In [136]:
#cheking the n_grams function
tokenizer1 = Tokenizer(num_words=None, char_level=True, lower=False) 

tokenizer1.fit_on_texts(train1) 
new_data1 = tokenizer.texts_to_sequences(train1) 


#N-Grams Sequence 
seqLen1 = 15 
stepSize = 1 
input_chars, next_char = n_grams(seqLen1, stepSize, new_data1)

In [137]:
x_train, y_train = preparation(train, tokenizer)

x_val, y_val = preparation(val, tokenizer)

In [138]:
x_train1, y_train1 = preparation(train1, tokenizer1)

x_val1, y_val1 = preparation(val1, tokenizer1)

In [139]:
vocab_size = len(tokenizer.word_index)

In [140]:
vocab_size1 = len(tokenizer1.word_index)

In [141]:
# preparing test data for alzheimer dataset
test2 = ''.join(test1) 

In [142]:
x_test, y_test = preparation(test2, tokenizer1)

In [143]:
def generation(seed,tokenizer,num_char,model):
    for i in range(num_char):
        tokens = tokenizer.texts_to_sequences([seed])[0]
        tokens = np.array(tokens)
        tokens = tokens[i:15+i]
        tokens = tokens.reshape(1,15)
        predicted = model.predict(tokens) #shape (1,43)
        ind = np.argmax(predicted)
    
        for word,index in tokenizer.word_index.items():
            predicted_word = ""
            if index == ind:
                predicted_word = word
                break
            if predicted_word == "£":
                break

        seed += predicted_word
    return seed


In [144]:
def checkSMILES(smiles):
	valid_smiles = []
	for smile in smiles:
		new = ''
		for char in smile:
			if char != '$' and char != '£':	
				new += char

		if Chem.MolFromSmiles(new, sanitize=False) is not None and len(new) > 15:
			valid_smiles.append(new)
			print(new)

	return valid_smiles